In [16]:
import pandas as pd
import mlflow
from sklearn.metrics import recall_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv(r"../data/gd.csv")

In [23]:
df.head()

,Account_status,Duration_months,Credit_history,loan_Purpose,Credit_amount,Savings/Bonds_AC,Present_Employment_since,loan_wage_ratio,Sex,Marital_status,...,Assets/Physical_property,Age,Other_loans,Housing,Existing Credit,Job_status,Dependents,Telephone,Foreign_worker,Credit_risk
0,A11,6,A34,A43,1169,A65,A75,4,Male,single,...,A121,67,A143,A152,2,A173,1,A192,A201,0
1,A12,48,A32,A43,5951,A61,A73,2,Female,divorced/separated/married,...,A121,22,A143,A152,1,A173,1,A191,A201,1
2,A14,12,A34,A46,2096,A61,A74,2,Male,single,...,A121,49,A143,A152,1,A172,2,A191,A201,0
3,A11,42,A32,A42,7882,A61,A74,2,Male,single,...,A122,45,A143,A153,1,A173,2,A191,A201,0
4,A11,24,A33,A40,4870,A61,A73,3,Male,single,...,A124,53,A143,A153,2,A173,2,A191,A201,1


In [22]:
df['Credit_risk'] = df['Credit_risk'].map({1:0, 2:1})

In [4]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [5]:
import mlflow.pyfunc

In [11]:
model_name = "german_bank_model"

alias = "pioneer"
model_uri = f"models:/{model_name}@{alias}"

In [12]:
from mlflow import MlflowClient

client = MlflowClient()

try:
    model_info = client.get_registered_model(model_name)
    print(f"Model Found: {model_info.name}")
    print(f"Raw Alias dictionary: {model_info.aliases}")
except Exception as e:
    print(e)

Model Found: german_bank_model
Raw Alias dictionary: {'champion': '1', 'pioneer': '2'}


In [13]:
loaded_model = mlflow.pyfunc.load_model(model_uri)

d:\Testing & Deployment\ml_env\lib\site-packages\mlflow\pyfunc\utils\data_validation.py:186: UserWarning: Add type hints to the `predict` method to enable data validation and automatic signature inference during model logging. Check https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage-in-pythonmodel for more details.
  color_warning(


In [24]:
X, y = df.iloc[:, :-1], df.iloc[:, -1]

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=27)

In [26]:
y_pred = loaded_model.predict(X_test)

In [27]:
confusion_matrix(y_test, y_pred)

array([[54, 90],
       [ 3, 53]])

In [28]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.38      0.54       144
           1       0.37      0.95      0.53        56

    accuracy                           0.54       200
   macro avg       0.66      0.66      0.53       200
weighted avg       0.79      0.54      0.54       200

